# **Module `home_credit.persist`**

✔ Revue des **typehints** et des **docstrings**.

**TODO** refactoring : déplacer cette librairie dans le package **`pepper`**.

# Mise au point d'une infrastructure de persistance

**Motivation** : pour que ce ne soit pas l’enfer des calculs d'agrégation et de fusion, il pouvoir calculer et sauvegarder toutes les tranches d’agrégation intermédiaires.

La question, c’est comment et quand choisir de charger des données sauvegardée ou de les mettre à jour à l'aide d'un nouveau calcul sous conditions changées (par exemple la précision).

**Spec** :

On se dote d'un mécanisme générique valable pour toutes les fonctions de chargement de données de la version objet des tables Home Credit, basé sur ces 3 paramètres optionnels de toutes les fonctions :
- **`from_file`** : s'il est fixé à vrai, il préempte le calcul et charge depuis le fichier, sous réserve que celui-ci soit disponible. Sinon, il opère comme **`update_file`**. Le nom du fichier n’est pas fourni, pas plus que le dossier de persistance qui sont fixés conventionnellement, cf. le cache de streamlit, à partir des paramètres de la fonction.
- **`no_cache`** pour indiquer qu’on ne souhaite pas surcharger le cache avec une table intermédiaire.
- **`update_file`** qui s’il est vrai entraîne la sauvegarde de ce qui va être calculé.

**Exemple d'application.** Cela pourrait être notamment utile pour des tables associatives simples comme les deux vecteurs de mappage utilisés par les fonctions `currentize` et `targetize` qui sont souvent appelées. À chaque utilisation après que le noyau ait redémarré, cela déclenche le chargement de la table `application` complète (ses deux parties `train` et `test` qui sont ensuite empilées verticalement), pour en extraire juste un petit sous-ensemble. Ce petit sous-ensemble stable, peu susceptible d’être modifié dans le futur a tout intérêt à reposer dans un fichier.

Les fichiers de sauvegarde sont au format `parquet`, ce qui permet d'obtenir des performances spatiales et temporelles significativement supérieures à celles du `CSV`.

Le dossier de persistance est arbitrairement fixé sur **`{project_path}/tmp/persist/`**.

Chaque table et ses dérivés sont regroupés dans un sous-dossier du nom de la table (et donc de la classe qui la représente) :
- **`{project_path}/tmp/persist/application/`**,
- **`{project_path}/tmp/persist/bureau/`**,
- **`{project_path}/tmp/persist/bureau_balance/`**,
- etc

Chaque fonction de production de table a son propre sous-dossier. Par exemple, les sorties de `Application.clean_extended()` et de `Application.clean()` seront sauvegardés dans les sous-dossiers :
- **`{project_path}/tmp/persist/application/clean/`**,
- **`{project_path}/tmp/persist/application/clean_extended/`**.

Par exception, les tables chargées avec la méthode `HomeCreditTable.raw()` le sont à partie du dossier **`{project_path}/dataset/pqt/`**. Cela reste l'emplacement des données de référence qui ne sont jamais modifiées, et il serait mal venu pour notre espace disque de les dupliquer dans **`{project_path}/tmp/persist/`**.

Les noms des fichiers `.pqt` sont déterminés à partir des valeurs des paramètres d'appel de la fonction.

Ainsi, une fonction sans paramètre n'aura au plus qu'un fichier sauvegardé.

L'encodage ? on s'inspire de l'encodage d'url ?


## Encodage des paramètres

Le but est de disposer d'une façon simple et générique pour obtenir le dossier et le code de hachage cible, associé avec les arguments d'appels de la fonction.

Cette version est insuffisamment sécurisée pour une librairie professionnelle, mais suffisante pour notre projet. Dans notre cas de figure, les seuls cas seront des fonctions ou des méthodes de classes.

Il y a ce qu'il faut d'introspection pour éviter de surcharger le travail et la vigilance du développeur. Une évolution pour une version future serait de renforcer les mécanismes d'introspection, et de rendre l'ajout encore plus léger avec un décorateur.

In [ ]:
from home_credit.persist import this_f_name, get_persist_params

class A:
    @classmethod
    def class_data_loader(cls, a, b=1, c=True, d=None):
        get_persist_params(locals().copy(), this_f_name(), True)

    @staticmethod
    def static_data_loader(a, b=1, c=True, d=None):
        get_persist_params(locals().copy(), this_f_name(), True)

    def instance_data_loader(self, a, b=1, c=True, d=None):
        get_persist_params(locals().copy(), this_f_name(), True)

def data_loader(a, b=1, c=True, d=None):
    get_persist_params(locals().copy(), this_f_name(), True)

A.class_data_loader(2, 3)
A.static_data_loader(2, 3)
A().instance_data_loader(2, 3)
data_loader(2, 3)

container class name: A
f_name: class_data_loader
kwargs: {'a': 2, 'b': 3, 'c': True, 'd': None}
hcode: ed4c450982bc4fc67b15499359a3b088ac04868d6fabc309e80062525be3a9e1
subdir: a
container class name: None
f_name: static_data_loader
kwargs: {'a': 2, 'b': 3, 'c': True, 'd': None}
hcode: ed4c450982bc4fc67b15499359a3b088ac04868d6fabc309e80062525be3a9e1
subdir: None
container class name: A
f_name: instance_data_loader
kwargs: {'a': 2, 'b': 3, 'c': True, 'd': None}
hcode: ed4c450982bc4fc67b15499359a3b088ac04868d6fabc309e80062525be3a9e1
subdir: a
container class name: None
f_name: data_loader
kwargs: {'a': 2, 'b': 3, 'c': True, 'd': None}
hcode: ed4c450982bc4fc67b15499359a3b088ac04868d6fabc309e80062525be3a9e1
subdir: None


## Gestion de l'index des dataframes sauvegardés

In [ ]:
from home_credit.persist import add_entry_to_index, _persist_index

add_entry_to_index("A", "load", {"a": 1, "b": None})
add_entry_to_index("B", "clean", {"a": 1, "b": None})
add_entry_to_index("B", "clean", {"a": 1, "b": 2})

display(_persist_index)

{'a': {'load': {'0c6b73e7b48e199ae73fa63c9220b7027a756e4a76995d767d9cf6fe0fbf6e88': {'a': 1,
    'b': None}}},
 'b': {'clean': {'0c6b73e7b48e199ae73fa63c9220b7027a756e4a76995d767d9cf6fe0fbf6e88': {'a': 1,
    'b': None},
   'd8497d9d82770a70729261095aa98f7ef5154d7af499f8037b6ca250296785a6': {'a': 1,
    'b': 2}}}}

## Utilisation

On réalise la partie la plus fonctionnelle de la spec.

3 paramètres optionnels de toutes les fonctions :
- **`from_file`** : s'il est fixé à vrai, il préempte le calcul et charge depuis le fichier, sous réserve que celui-ci soit disponible. Sinon, il opère comme **`update_file`**. Le nom du fichier n’est pas fourni, pas plus que le dossier de persistance qui sont fixés conventionnellement, cf. le cache de streamlit, à partir des paramètres de la fonction.
- **`no_cache`** pour indiquer qu’on ne souhaite pas surcharger le cache avec une table intermédiaire.
- **`update_file`** qui s’il est vrai entraîne la sauvegarde de ce qui va être calculé.

Exemple avec `Bureau.clean()`

Commençons par sauvegarder le résultat pour nous assurer du mécanisme de persistence indexée :

In [1]:
from home_credit.tables import Bureau

data = Bureau.clean()
display(data)

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\bureau.pqt
load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\application_train.pqt
load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\application_test.pqt


CLEAN_BUREAU             TARGET CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
SK_ID_CURR SK_ID_BUREAU                                                      
215354     5714462            0        Closed      currency 1          497   
           5714463            0        Active      currency 1          208   
           5714464            0        Active      currency 1          203   
           5714465            0        Active      currency 1          203   
           5714466            0        Active      currency 1          629   
...                         ...           ...             ...          ...   
259355     5057750            1        Active      currency 1           44   
100044     5057754            0        Closed      currency 1         2648   
           5057762            0        Closed      currency 1         1809   
246829     5057770            0        Closed      currency 1         1878   
           5057778            0        Closed      currency 1          463   

CLEAN_BUREAU             CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  \
SK_ID_CURR SK_ID_BUREAU                                            
215354     5714462                        0                153.0   
           5714463                        0              -1075.0   
           5714464                        0               -528.0   
           5714465                        0                  NaN   
           5714466                        0              -1197.0   
...                                     ...                  ...   
259355     5057750                        0                 30.0   
100044     5057754                        0               2433.0   
           5057762                        0               1628.0   
246829     5057770                        0               1513.0   
           5057778                        0                  NaN   

CLEAN_BUREAU             DAYS_ENDDATE_FACT  AMT_CREDIT_MAX_OVERDUE  \
SK_ID_CURR SK_ID_BUREAU                                              
215354     5714462                   153.0                     NaN   
           5714463                     NaN                     NaN   
           5714464                     NaN                     NaN   
           5714465                     NaN                     NaN   
           5714466                     NaN                 77674.5   
...                                    ...                     ...   
259355     5057750                     NaN                     0.0   
100044     5057754                  2493.0                  5476.5   
           5057762                   970.0                     NaN   
246829     5057770                  1513.0                     NaN   
           5057778                   387.0                     NaN   

CLEAN_BUREAU             CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
SK_ID_CURR SK_ID_BUREAU                                       
215354     5714462                        0        91323.00   
           5714463                        0       225000.00   
           5714464                        0       464323.50   
           5714465                        0        90000.00   
           5714466                        0      2700000.00   
...                                     ...             ...   
259355     5057750                        0        11250.00   
100044     5057754                        0        38130.84   
           5057762                        0        15570.00   
246829     5057770                        0        36000.00   
           5057778                        0        22500.00   

CLEAN_BUREAU             AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  \
SK_ID_CURR SK_ID_BUREAU                                              
215354     5714462                       0.0                   NaN   
           5714463                  171342.0                   NaN   
           5714464                       NaN                   NaN   
           5714465 

Sauvegarde :

In [4]:
from home_credit.persist import save_to_parquet, add_entry_to_index

data_build_params = {
    "class_name": "Bureau",
    "method_name": "clean",
    "arguments": {}
}
save_to_parquet(data, **data_build_params)
add_entry_to_index(**data_build_params)

Save to C:/Users/franc/Projects/pepper_credit_scoring_tool\tmp\persist\bureau\clean\44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a.pqt


Chargement :

In [1]:
from home_credit.persist import load_from_parquet, is_in_index

data_build_params = {
    "class_name": "Bureau",
    "method_name": "clean",
    "arguments": {}
}
if is_in_index(**data_build_params):
    data = load_from_parquet(**data_build_params)

display(data)

CLEAN_BUREAU             TARGET CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
SK_ID_CURR SK_ID_BUREAU                                                      
215354     5714462            0        Closed      currency 1          497   
           5714463            0        Active      currency 1          208   
           5714464            0        Active      currency 1          203   
           5714465            0        Active      currency 1          203   
           5714466            0        Active      currency 1          629   
...                         ...           ...             ...          ...   
259355     5057750            1        Active      currency 1           44   
100044     5057754            0        Closed      currency 1         2648   
           5057762            0        Closed      currency 1         1809   
246829     5057770            0        Closed      currency 1         1878   
           5057778            0        Closed      currency 1          463   

CLEAN_BUREAU             CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  \
SK_ID_CURR SK_ID_BUREAU                                            
215354     5714462                        0                153.0   
           5714463                        0              -1075.0   
           5714464                        0               -528.0   
           5714465                        0                  NaN   
           5714466                        0              -1197.0   
...                                     ...                  ...   
259355     5057750                        0                 30.0   
100044     5057754                        0               2433.0   
           5057762                        0               1628.0   
246829     5057770                        0               1513.0   
           5057778                        0                  NaN   

CLEAN_BUREAU             DAYS_ENDDATE_FACT  AMT_CREDIT_MAX_OVERDUE  \
SK_ID_CURR SK_ID_BUREAU                                              
215354     5714462                   153.0                     NaN   
           5714463                     NaN                     NaN   
           5714464                     NaN                     NaN   
           5714465                     NaN                     NaN   
           5714466                     NaN                 77674.5   
...                                    ...                     ...   
259355     5057750                     NaN                     0.0   
100044     5057754                  2493.0                  5476.5   
           5057762                   970.0                     NaN   
246829     5057770                  1513.0                     NaN   
           5057778                   387.0                     NaN   

CLEAN_BUREAU             CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
SK_ID_CURR SK_ID_BUREAU                                       
215354     5714462                        0        91323.00   
           5714463                        0       225000.00   
           5714464                        0       464323.50   
           5714465                        0        90000.00   
           5714466                        0      2700000.00   
...                                     ...             ...   
259355     5057750                        0        11250.00   
100044     5057754                        0        38130.84   
           5057762                        0        15570.00   
246829     5057770                        0        36000.00   
           5057778                        0        22500.00   

CLEAN_BUREAU             AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  \
SK_ID_CURR SK_ID_BUREAU                                              
215354     5714462                       0.0                   NaN   
           5714463                  171342.0                   NaN   
           5714464                       NaN                   NaN   
           5714465 

Révisons nos classes pour intégrer ces mécanismes : la méthode **`home_credit.persist.controlled_load`** termine l'interface utilisateur et réalise la stratégie spécifiée, avec un maximum de simplicité d'utilisation.

In [1]:
from home_credit.tables import Bureau

data = Bureau.clean()
display(data)

container class name: Bureau
f_name: clean
kwargs: {}
hcode: 44136fa355b3678a1146ad16f7e8649e94fb4fc21fe77e8310c060f61caaff8a
subdir: bureau


CLEAN_BUREAU             TARGET CREDIT_ACTIVE  ... DAYS_CREDIT_UPDATE  AMT_ANNUITY
SK_ID_CURR SK_ID_BUREAU                        ...                                
215354     5714462            0        Closed  ...                131          NaN
           5714463            0        Active  ...                 20          NaN
           5714464            0        Active  ...                 16          NaN
           5714465            0        Active  ...                 16          NaN
           5714466            0        Active  ...                 21          NaN
...                         ...           ...  ...                ...          ...
259355     5057750            1        Active  ...                 19          NaN
100044     5057754            0        Closed  ...               2493          NaN
           5057762            0        Closed  ...                967          NaN
246829     5057770            0        Closed  ...               1508          NaN
           5057778            0        Closed  ...                387          NaN

[1716428 rows x 16 columns]

In [4]:
from home_credit.persist import get_persist_params
from home_credit.tables import Bureau

kwargs = {"cls": Bureau, "alpha": 1, "decimals": 2, "include_rle": True}
method = "extended_clean"
debug = True

params = get_persist_params(kwargs, method, debug)

container class name: Bureau
f_name: extended_clean
kwargs: {'alpha': 1, 'decimals': 2, 'include_rle': True}
hcode: 303ecaaa875cb35a4fb4805fc6aabbdeedea738ec189588b6daeef892e5de2ec
subdir: bureau


# Persistance Parquet de cellules contenant des `n-ndarrays`

Parquet convertit les structures multi dimensionnelles en `1-ndarrays` imbriqués (la forme native du langage C).

Malheureusement, cette conversion allonge considérablement le temps de sauvegarde.

Pire, le coût de chargement, et de l'éventuelle reconstruction de la structure multi-dimensionnelle d'origine est prohibitif au point de faire perdre son intérêt à une persistence Parquet.

Nous avons donc développé une persistence ad hoc pour les structures multi-dimensionnelles, déjà explicitement converties en `ndarrays`. On laisse à Parquet son comportement par défaut pour des listes ou des tuples multidimensionnels.

La stratégie consiste à aplatir les tableaux à l'aide de `np.ravel` après avoir obtenu et mémorisé la forme à l'aide de `np.shape`. Au chargement, on peut ainsi redonner sa forme à la structure, en tant que vue, à l'aide de `np.reshape`.

## Approche par aplatissement

L'idée est de gérer d'enregistrer une version aplatie et de charger et recomposer avec `np.reshape`.

Il faut donc :
- au moment de l'enregistrement, encoder chaque colonne `A` de `ndarrays` en un couple `A_shape`, `A_flat`.
- au moment du chargement, il faut décoder `A` à partir de ces deux informations.

**NB** la structure rechargée reste fondamentalement 1d, mais elle se présente sous la forme d'une vue nd. Loin d'être un problème, c'est une optimisation souhaitable pour les performances.

**TODO** retravailler l'espace des exemples et des tests.

### Cas simple

#### Récupération des données brutes nd

In [1]:
from home_credit.tables import BureauBalance

status_variation = BureauBalance.rle_loan_status_variation()
display(status_variation)

BUREAU_LOAN_STATUS_BY_MONTH_VARIATIONS,STATUS
SK_ID_BUREAU,
5001710,"[[0, 83]]"
5001711,"[[0, 4]]"
5001712,"[[0, 19]]"
5001713,"[[0, 22]]"
5001714,"[[0, 15]]"
...,...
6842884,"[[0, 48]]"
6842885,"[[5, 12], [0, 12]]"
6842886,"[[0, 33]]"


#### Encodage et sauvegarde

In [5]:
import numpy as np

shape = status_variation.STATUS.apply(np.shape).rename("STATUS_shape")
display(shape)

SK_ID_BUREAU
5001710    (1, 2)
5001711    (1, 2)
5001712    (1, 2)
5001713    (1, 2)
5001714    (1, 2)
            ...  
6842884    (1, 2)
6842885    (2, 2)
6842886    (1, 2)
6842887    (1, 2)
6842888    (3, 2)
Name: STATUS_shape, Length: 774354, dtype: object

In [6]:
import numpy as np

flat = status_variation.STATUS.apply(np.ravel).rename("STATUS_flat")
display(flat)

SK_ID_BUREAU
5001710                [0, 83]
5001711                 [0, 4]
5001712                [0, 19]
5001713                [0, 22]
5001714                [0, 15]
                  ...         
6842884                [0, 48]
6842885         [5, 12, 0, 12]
6842886                [0, 33]
6842887                [0, 37]
6842888    [0, 58, 1, 1, 0, 3]
Name: STATUS_flat, Length: 774354, dtype: object

In [7]:
import pandas as pd

encoded_status = pd.concat([shape, flat], axis=1)
display(encoded_status)

,STATUS_shape,STATUS_flat
SK_ID_BUREAU,,
5001710,"(1, 2)","[0, 83]"
5001711,"(1, 2)","[0, 4]"
5001712,"(1, 2)","[0, 19]"
5001713,"(1, 2)","[0, 22]"
5001714,"(1, 2)","[0, 15]"
...,...,...
6842884,"(1, 2)","[0, 48]"
6842885,"(2, 2)","[5, 12, 0, 12]"
6842886,"(1, 2)","[0, 33]"


In [8]:
# Save the DataFrame to Parquet
file_path = "encoded_status.pqt"
print(f"Save to {file_path}")
encoded_status.to_parquet(file_path, engine="pyarrow", compression="gzip")

Save to encoded_status.pqt


#### Chargement et décodage

In [9]:
# Load the data from the Parquet file
loaded_data = pd.read_parquet(file_path, engine="pyarrow")
display(loaded_data)

,STATUS_shape,STATUS_flat
SK_ID_BUREAU,,
5001710,"[1, 2]","[0, 83]"
5001711,"[1, 2]","[0, 4]"
5001712,"[1, 2]","[0, 19]"
5001713,"[1, 2]","[0, 22]"
5001714,"[1, 2]","[0, 15]"
...,...,...
6842884,"[1, 2]","[0, 48]"
6842885,"[2, 2]","[5, 12, 0, 12]"
6842886,"[1, 2]","[0, 33]"


In [19]:
loaded_data_short = loaded_data.head(5).copy()

def decode_ndarray(row):
    shape = row[0]
    flat = row[1]
    return flat.reshape(shape)

loaded_data_short = loaded_data_short.apply(decode_ndarray, axis=1)

display(loaded_data_short)

SK_ID_BUREAU
5001710    [[0, 83]]
5001711     [[0, 4]]
5001712    [[0, 19]]
5001713    [[0, 22]]
5001714    [[0, 15]]
dtype: object

In [22]:
coded_columns = ["STATUS_shape", "STATUS_flat"]
loaded_data["STATUS"] = loaded_data[coded_columns].apply(decode_ndarray, axis=1)
decoded_status = loaded_data[["STATUS"]]
display(decoded_status)

,STATUS
SK_ID_BUREAU,
5001710,"[[0, 83]]"
5001711,"[[0, 4]]"
5001712,"[[0, 19]]"
5001713,"[[0, 22]]"
5001714,"[[0, 15]]"
...,...
6842884,"[[0, 48]]"
6842885,"[[5, 12], [0, 12]]"
6842886,"[[0, 33]]"


#### Version intégrée

In [7]:
from home_credit.persist import decode_ndarrays_from_parquet
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'A_shape': [(2, 3), (3, 2)],
    'A_flat': [
        np.array([1, 2, 3, 4, 5, 6]),
        np.array([7, 8, 9, 10, 11, 12])
    ]
})
decoded_data = decode_ndarrays_from_parquet(data)
display(decoded_data)

,A
0,"[[1, 2, 3], [4, 5, 6]]"
1,"[[7, 8], [9, 10], [11, 12]]"


### Cas complexe

C'est le cas qui a révélé le problème de performance ici traité.

On effectue un test complet avec expansion RLE, NaN compris.

In [1]:
from home_credit.tables import Bureau

extended_clean_bureau = Bureau.extended_clean(include_rle=True)
display(extended_clean_bureau)

TARGET CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
SK_ID_CURR SK_ID_BUREAU                                                      
100001     5896630           -1        Closed      currency 1          857   
           5896631           -1        Closed      currency 1          909   
           5896632           -1        Closed      currency 1          879   
           5896633           -1        Closed      currency 1         1572   
           5896634           -1        Active      currency 1          559   
...                         ...           ...             ...          ...   
456255     5126333            0        Active      currency 1          363   
           5126334            0        Active      currency 1          451   
           5126335            0        Closed      currency 1         1512   
           5126336            0        Closed      currency 1         2337   
           5126337            0        Closed      currency 1         2337   

                         CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  \
SK_ID_CURR SK_ID_BUREAU                                            
100001     5896630                        0                492.0   
           5896631                        0                179.0   
           5896632                        0                514.0   
           5896633                        0               1329.0   
           5896634                        0               -902.0   
...                                     ...                  ...   
456255     5126333                        0              -1463.0   
           5126334                        0               -279.0   
           5126335                        0               -315.0   
           5126336                        0              -8620.0   
           5126337                        0               1243.0   

                         DAYS_ENDDATE_FACT  AMT_CREDIT_MAX_OVERDUE  \
SK_ID_CURR SK_ID_BUREAU                                              
100001     5896630                   553.0                     NaN   
           5896631                   877.0                     NaN   
           5896632                   544.0                     NaN   
           5896633                  1328.0                     NaN   
           5896634                     NaN                     NaN   
...                                    ...                     ...   
456255     5126333                     NaN                   0.000   
           5126334                     NaN               15439.905   
           5126335                   781.0               25578.000   
           5126336                   779.0                2646.000   
           5126337                  1621.0               16618.500   

                         CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  ...  \
SK_ID_CURR SK_ID_BUREAU                                      ...   
100001     5896630                        0        112500.0  ...   
           5896631                        0        279720.0  ...   
           5896632                        0         91620.0  ...   
           5896633                        0         85500.0  ...   
           5896634                        0        337680.0  ...   
...                                     ...             ...  ...   
456255     5126333                        0        436032.0  ...   
           5126334                        0        450000.0  ...   
           5126335                        0        900000.0  ...   
           5126336                        1         38925.0  ...   
           5126337                        0        450000.0  ...   

                         AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
SK_ID_CURR SK_ID_BUREAU                                                 
100001     5896630                        0.0                     0.0   
           5896631                        0.0                     0.0   
           5896632                   

In [2]:
from pepper.rle import rle_expand_dataframe

support = extended_clean_bureau.MONTH_support
frame = extended_clean_bureau.ACTIVITY_frame

exp_rle = rle_expand_dataframe(support, frame)
display(exp_rle)

0    1    2    3    4    5    6    7    8    9   \
SK_ID_CURR SK_ID_BUREAU                                                     
100001     5896630       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           5896631       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           5896632       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           5896633       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           5896634       1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
...                      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
456255     5126333       0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
           5126334       0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0   
           5126335       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
           5126336       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
           5126337       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

                         ...  87  88  89  90  91  92  93  94  95  96  
SK_ID_CURR SK_ID_BUREAU  ...                                          
100001     5896630       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5896631       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5896632       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5896633       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5896634       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
...                      ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  
456255     5126333       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5126334       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5126335       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5126336       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
           5126337       ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[1716428 rows x 97 columns]